In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import networkx as nx

# Read data

In [2]:
# Read dictionary with municipalities
muni = pd.read_csv('/home/juliane.oliveira/Documents/AESOP data structure/ETL_data/DTB_BRASIL_MUNICIPIO.csv',sep = ';')
    

In [3]:
muni = muni[['UF', 'Nome_UF', 'Mesorregião Geográfica', 'Nome_Mesorregião',
       'Microrregião Geográfica', 'Nome_Microrregião', 'Município',
       'Código Município Completo', 'Nome_Município']]

In [4]:
# Read the Adjacent matrix
link0 = '/home/juliane.oliveira/Documents/GitHub/aesop-models/scripts/Mobility pipeline/adjacency_matrix_correct.parquet'

matrix = pd.read_parquet(link0, engine='pyarrow')

In [5]:
df_np = matrix.to_numpy()

In [6]:
hubs = pd.read_csv('/home/juliane.oliveira/workspace/Data/data/lists_of_hubs.csv')

In [7]:
hub_pop_density = pd.read_csv('/home/juliane.oliveira/workspace/Data/hub_pop_density.csv')

# Functions

In [8]:
link_muni_vertice = pd.DataFrame(matrix.columns, columns=['muni'])

In [9]:
def get_mname(n):
    
    m = link_muni_vertice.iloc[n]['muni']
    set_muni = muni[muni['Código Município Completo'] == m].reset_index()
    return [set_muni.iloc[0]['Nome_Município'],set_muni.iloc[0]['Nome_UF'],m]
   
#def get_mnumber(name):
#    muni[muni['Nome_Município'] == name]
#    
#    co_mu = muni[muni['Nome_Município'] == name].reset_index()['Código Município Completo'][0]
#    muni_number = link_muni_vertice[link_muni_vertice['muni'] == co_mu]['muni'].index.tolist()[0]
#    return [muni_number, co_mu]

def get_mnumber(name,uf):
    
    set_df = muni[(muni['Nome_Município'] == name) & (muni['Nome_UF'] == uf)].reset_index()
    
    co_mu = set_df['Código Município Completo'][0]
    
    muni_number = link_muni_vertice[link_muni_vertice['muni'] == co_mu]['muni'].index.tolist()[0]
    
    return [muni_number, co_mu]

In [10]:
def get_mnumber_of_set(df_col):
    muni_number = []

    for i in range(0,len(df_col)):
    
        uf = df_col['Nome_UF'].iloc[i]
    
        city = df_col['Nome_Município'].iloc[i]
    
        muni_number.append(get_mnumber(city,uf)[0])
    
    return muni_number

In [11]:
# Python program for implementation
# of Ford Fulkerson algorithm
from collections import defaultdict

# This class represents a directed graph
# using adjacency matrix representation
        
class Graph:

    def __init__(self, graph):
        self.graph = graph # residual graph
        self. ROW = len(graph)
        # self.COL = len(gr[0])


    '''Returns true if there is a path from source 's' to sink 't' in
    residual graph. Also fills parent[] to store the path '''


    def BFS(self, s, t, parent):
        visited = [False] * self.ROW
        queue = []
        queue.append(s)
        visited[s] = True

        while queue:
            u = queue.pop(0)
            for ind, val in enumerate(self.graph[u]):
                if visited[ind] == False and val > 0:
                    queue.append(ind)
                    visited[ind] = True
                    parent[ind] = u
                    if ind == t:
                        return True
        return False

    def FordFulkerson(self, source, sink):
        parent = [-1] * self.ROW
        max_flow = 0
        paths = []  # List to store the paths
        value_path = []

        while self.BFS(source, sink, parent):
            path_flow = float("inf")
            s = sink
            while s != source:
                path_flow = min(path_flow, self.graph[parent[s]][s])
                s = parent[s]

            max_flow += path_flow

            # Store the current path
            current_path = []
            v = sink
            while v != source:
                u = parent[v]
                current_path.append((u, v))
                self.graph[u][v] -= path_flow
                self.graph[v][u] += path_flow
                v = parent[v]
            paths.append(current_path)
            value_path.append(max_flow)
            
            if len(current_path) > 3:
                break

        df_paths = pd.DataFrame(paths).transpose()
        return max_flow, df_paths,value_path

# Define the list of municipalities as the origen and destination to run the FF method

## Destination 

In [12]:
hub_pop_density = hub_pop_density[[ '_c0', 'Nome_UF','UF',  'Nome_Município', 'uf10', 'co_uf',
       'nm_municipio', 'ind_proxi', 'ind_intermed',  'densidade_2022', 
       'co_ibge']]

In [27]:
#hub_pop_density.head()

In [28]:
#hub_pop_density.ind_proxi.describe()

In [22]:
# Select cities that are the top in PageRank and not in top of the BI

set_dest = hub_pop_density[(hub_pop_density.ind_intermed < 0.000053) & (hub_pop_density.ind_proxi >= 0.36)]

In [26]:
#set_dest

## Select origin cities for the sensitive analysis 

In [24]:
# Use the calculated probabilities to randomly select a subset of municipalities
subset_size = 100  # Adjust the subset size as needed
subset = np.random.choice(hub_pop_density['_c0'], size=subset_size, replace=False)


# Run method for the list of municipalities selected

In [25]:
lst1 = list(subset)
lst2 = list(set_dest['_c0']) # destination are the cities in the top of PR that are not in the BI

print(len(lst1), ',', len(lst2), ',', len(lst1)*len(lst2))

100 , 423 , 42300


In [133]:
# Define a function to process each value in parallel
def process_value(value):
    n, m = value

    matrix = pd.read_parquet(link0, engine='pyarrow')
    df_np = matrix.to_numpy()

    graph = df_np
    g = Graph(graph)

    max_flow, df_paths, value_path = g.FordFulkerson(n, m)
    # transform df int a variable of lists
    lists = [df_paths[col].tolist() for col in df_paths]
    
    #format value_path
    path_value = pd.DataFrame(value_path, columns=['value_cum'])
    path_value = path_value.assign(value=path_value['value_cum'] - path_value['value_cum'].shift())
    path_value['value'] = path_value['value'].fillna(0)
    path_value['ori_muni_name'] = get_mname(n)[0]
    path_value['ori_uf_name'] = get_mname(n)[1]
    path_value['ori_co_ibge'] = get_mname(n)[2]
    path_value['des_muni_name'] = get_mname(m)[0]
    path_value['des_uf_name'] = get_mname(m)[1]
    path_value['des_co_ibge'] = get_mname(m)[2]
    
    #add lists to path_value
    path_value = path_value.assign(paths=lists)
    
    #save path_value
    table2 = pa.Table.from_pandas(path_value)

    string2 = str(get_mname(n)[2]) + '_' + str(get_mname(m)[2]) + '_' + 'path_value'

    path_to_save2 = '/home/juliane.oliveira/workspace/Data/sent_path_FF_Proxi_100_423/{}.parquet'.format(string2)
    
    pq.write_table(table2, path_to_save2)
    

In [ ]:
import concurrent.futures
# import timeit
# start_time = timeit.default_timer()

# Create a list of values to process
values_to_process = [(x, y) for x in lst1 for y in lst2]

# Create a ThreadPoolExecutor to run the processing function in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(process_value, values_to_process)
    
# elapsed = timeit.default_timer() - start_time
# print("Time: " + str(elapsed))